In [2]:
import os

import xarray as xr
import pandas as pd

# User input

In [5]:
# Filepath to river metadata csvs
real_riv_metadata = r"../data/real_river_metadata.csv"
riv_no_mapping_fpath = (
    r"/home/jovyan/shared/common/oslofjord_modelling/oslofjord_location_ids.csv"
)

# Filepaths to WWTW data
isource_oslofjord_ids = list(range(1, 14)) + [23]  # Just directly inputting to Oslofjord
wwtp_fpath = r'/home/jovyan/shared/common/oslofjord_modelling/MARTINI800v10_river_inputs/of800_input_files/of800_v10b_isource_1990_2022_baseline.nc'
wwtp_isource_map_fpath = r'/home/jovyan/shared/common/oslofjord_modelling/MARTINI800v10_river_inputs/Flux_Martini_Rivers_sewage_with_mapping_and_fractions.xlsx'
sheet = 'vassområde_link'
riv_additions_sheet = 'riv_additions'

# Start and end years to summarise data over
start_year = 2018  # 2016
end_year = 2018

# Folders for output files
csv_exploration_folder = r"../output/csvs"
fig_folder = r"../output/plots"

# River numbers to use
oslofjord_riv_nos = range(6, 21)  # Real river numbers
# (Model numbers 5 to 25 incl.)

# Set up & read in river metadata

In [3]:
# River chemistry metadata
river_meta_df = pd.read_csv(real_riv_metadata, index_col=0, dtype={'Vassom':str})
# Limit to just Oslofjord rivers
river_meta_df = river_meta_df[river_meta_df.index.isin(oslofjord_riv_nos)]
# Add 'total' row for use later
river_meta_df.loc['Total', ['river_name', 'Regine', 'Regine_to_sea', 'Vassom']] = 'Total'
river_meta_df.query('real_river in @oslofjord_riv_nos')

,river_name,Outflow_lat,Outflow_lon,Regine,Regine_to_sea,Vassom,Vassom_area_land,Vassom_area_tot,Andre_MCA_area,Andre_area_q,Overestimate (%),Comment
real_river,,,,,,,,,,,,
6,Tista,59.119,11.370,001.A1,001.A1,001.,2495.0,2507.0,2507.0,1584.0,0.0,NaN
7,Glomma,59.206,10.953,002.A51,002.A11,002.,42446.0,43116.0,43116.0,41967.0,2.0,Monitoring point upstream of Sarpsborg RA. So ...
8,Mosseelva,59.439,10.662,003.A1,003.A1,003.,854.0,1052.0,1054.0,694.0,23.0,NaN
9,Hølenelva,59.523,10.690,004.A0,004.A0,004.,204.0,227.0,NaN,NaN,11.0,NaN
10,Årungen,59.720,10.728,005.3A,005.3A,005.,280.0,368.0,144.0,85.0,31.0,NaN
11,Akerselva,59.908,10.756,006.A10,006.A10,006.,392.0,415.0,415.0,307.0,6.0,NaN
12,Lysakerelva,59.914,10.640,007.A0,007.A0,007.,202.0,211.0,211.0,177.0,4.0,NaN
13,Sandvikselva,59.890,10.523,008.A11,008.A11,008.,279.0,311.0,311.0,226.0,11.0,NaN
14,Åros,59.704,10.519,009.A0,009.A0,009.,215.0,253.0,NaN,NaN,18.0,NaN


## Mapping

In [4]:
isource_vassom_map_df = pd.read_excel(wwtp_isource_map_fpath, sheet_name=sheet, index_col=0)
# Reformat vassdragsområde integer to match string format used elsewhere
isource_vassom_map_df['Vassdragsområde'] = isource_vassom_map_df['Vassdragsområde'].apply(lambda x: (str(x)).zfill(3) + '.')
isource_vassom_map_df

,Vassdragsområde,Comment
iSource,,
1,003.,NaN
2,003.,NaN
3,003.,NaN
4,004.,NaN
5,005.,NaN
6,006.,NaN
7,009.,VEAS main outlet
23,007.,VEAS overflow outlet. TEOTIL probably thinks t...
8,011.,NaN


In [5]:
# Mapping from real river number to model river number
# (index=model river ID, returns real river)
river_ns_s = pd.read_csv(riv_no_mapping_fpath, index_col=0).squeeze('columns')

# Read in direct sewage inputs

Calculate the total wastewater treatment plant N loads that are included in OF800 as direct inputs to the fjord. This is implemented either by adding loads to the model river fluxes (PWA does this as a post-processing step) or adding loads directly to the fjord (the iSources netcdf file).

In [6]:
# Direct point source inputs to include in the domain
# (see Phil's map for location of ids)

def isource_to_df(wwtp_fpath):
    """
    Input: filepath to ERSEN internal sources netcdf
    Output: Dataframe of monthly fluxes (units T/month)
    Read in ERSEM internal source netcdf.
    Only keep N data, convert units to mg/l, drop depth info,
    calculate total N and DIN, save as dataframe
    """

    # Molecular weight of N (g/mole)
    Mr_N = 14.0067

    # WWTP direct inputs
    wwtp_ds = xr.open_dataset(wwtp_fpath)

    # Sum over the 's_rho' dimension
    wwtp_ds = wwtp_ds.sum(dim='s_rho')

    wwtp_df = wwtp_ds.to_dataframe()
    wwtp_df = wwtp_df.reset_index()  # Drop multiindex
    wwtp_ds.close()

    # Drop all columns which are not N fluxes
    cols_to_keep = ['isource', 'isource_time'] + [col for col in wwtp_df.columns if col.endswith('_n')]
    wwtp_df = wwtp_df[cols_to_keep]

    # Drop all WWTPs which are outside oslofjord
    # wwtp_df = wwtp_df[wwtp_df['isource'].isin(isource_oslofjord_ids)]

    wwtp_df['isource_time'] = pd.to_datetime(wwtp_df['isource_time'])

    print('Number of WWTPs: %s ' % len(wwtp_df['isource'].drop_duplicates()))

    # Calculate TOTN
    cols_to_exclude = ['isource', 'isource_time']
    cols_to_sum = [col for col in cols_to_keep if col not in cols_to_exclude]

    # Create a new column 'sum' that is the sum of all columns in cols_to_sum
    wwtp_df['TOTN'] = wwtp_df[cols_to_sum].sum(axis=1)
    wwtp_df['DIN'] = wwtp_df['isource_N3_n'] + wwtp_df['isource_N4_n']

    wwtp_chem_vars = ['TOTN', 'DIN']

    # Drop all the subfractions apart from TOTN and DIN
    wwtp_df = wwtp_df.drop(cols_to_sum, axis=1)

    # Convert units from mmol/s to 10^3 kg/month (tonnes/month)
    # NB NEEDS IMPROVING, not accurate now, uses average number of days/month
    # instead of actual number of days/month
    for var in ['TOTN', 'DIN']:
        wwtp_df[var] = wwtp_df[var] * Mr_N * 86400 * 30.417 * 10**-9

    return wwtp_df


def calculate_average_annual_sum(df, start_year, end_year):
    # Pick out years within the user-specified range
    df['year'] = df['isource_time'].dt.year
    df = df[(df['year'] >= start_year) & (df['year'] <= end_year)].copy()

    # Calculate annual sums for all the remaining years
    df = df[['TOTN', 'DIN', 'isource', 'year']].groupby(['isource', 'year']).sum().reset_index()

    # Average the annual sums to get a single value per isource
    df = df.groupby('isource').mean()

    # Drop 'year' column
    df = df[['TOTN', 'DIN']]

    return df


def sum_over_vassom(df):
    """
    Sum up multiple iSource inputs per vassdragsområde
    Input: dataframe, index=iSource id, float cols to be summed
    Output: dataframe, index=Vassdragsområde (format '00x.'), same cols as input df
            units T/yr
    """
    df['Vassom'] = df.index.map(isource_vassom_map_df['Vassdragsområde'])

    # Drop rows where Vassom == NaN (further SW than we are considering here)
    df = df.dropna(how='any', axis=0)

    df = df.groupby('Vassom').sum()

    return df


def read_wwtp_river_inputs(fpath, sheet):
    """
    Some of the WWTWs around Oslofjord are added to the river input file rather
    than as iSources. Read in those data here and sum them up over vassdragsområder
    Input: filepath to excel file and sheet name
    Output: dataframe, index: Vassdragsområde number (format '00x.'), fluxes in T/yr
    """
    wwtp_riv_df = pd.read_excel(fpath, sheet)
    wwtp_riv_df = wwtp_riv_df[['model_riv_id', 'TN_total_ton_yr', 'DIN_total_ton_yr']]

    # Map model river numbers to real river numbers
    wwtp_riv_df['real_riv_id'] = wwtp_riv_df['model_riv_id'].map(river_ns_s)

    # Add vassdragsområde
    wwtp_riv_df['Vassom'] = wwtp_riv_df['real_riv_id'].map(river_meta_df['Vassom'])

    # Sum up multiple inputs per Vassom
    wwtp_riv_df = wwtp_riv_df[['TN_total_ton_yr', 'DIN_total_ton_yr', 'Vassom']].groupby('Vassom').sum()

    # Match column names to isource dataframe
    wwtp_riv_df.columns = ['TOTN', 'DIN']

    return wwtp_riv_df


def total_wwtw_inputs(isource_df, wwtp_riv_df, units='T'):
    """
    Sum direct WWTW inputs from the iSource file and inputs added to
    the river file to get total extra WWTW additions to Oslofjord
    Input: units can be 'T' or 'kg'
    """

    direct_wwtp_annual_df = wwtp_riv_df.add(isource_df, fill_value=0)

    # Also add in mapping to real_river, for compatibility with work below
    mapping_dict = river_meta_df.reset_index().set_index('Vassom')['real_river'].to_dict()
    direct_wwtp_annual_df['real_river'] = direct_wwtp_annual_df.index.map(mapping_dict)

    # Drop NaNs (Larvik in this case)
    direct_wwtp_annual_df = direct_wwtp_annual_df.dropna(how='any', axis=0)

    # Optionally convert units to kg
    if units == 'kg':
        direct_wwtp_annual_df[['TOTN', 'DIN']] = round((direct_wwtp_annual_df[['TOTN', 'DIN']] * 1000), 2)

    return direct_wwtp_annual_df

In [6]:
wwtp_ds = xr.open_dataset(wwtp_fpath)
wwtp_ds

<xarray.Dataset> Size: 105MB
Dimensions:            (isource: 23, isource_time: 398, s_rho: 42)
Coordinates:
  * isource            (isource) float64 184B 1.0 2.0 3.0 4.0 ... 21.0 22.0 23.0
  * isource_time       (isource_time) datetime64[ns] 3kB 1989-12-15 ... 2023-...
Dimensions without coordinates: s_rho
Data variables: (12/36)
    isource_Xposition  (isource) float64 184B ...
    isource_Eposition  (isource) float64 184B ...
    isource_CR1_c      (isource_time, s_rho, isource) float64 3MB ...
    isource_CR1_n      (isource_time, s_rho, isource) float64 3MB ...
    isource_CR1_p      (isource_time, s_rho, isource) float64 3MB ...
    isource_CR2_c      (isource_time, s_rho, isource) float64 3MB ...
    ...                 ...
    isource_R6_c       (isource_time, s_rho, isource) float64 3MB ...
    isource_R6_n       (isource_time, s_rho, isource) float64 3MB ...
    isource_R6_p       (isource_time, s_rho, isource) float64 3MB ...
    isource_R8_c       (isource_time, s_rho, isource) float64 3MB ...
    isource_R8_n       (isource_time, s_rho, isource) float64 3MB ...
    isource_R8_p       (isource_time, s_rho, isource) float64 3MB ...
Attributes:
    history:  Sat Dec 16 21:05:48 2023: ncap2 -s isource_Xposition=isource_Xp...
    type:     ROMS isource forcing file
    NCO:      netCDF Operators version 5.1.3 (Homepage = http://nco.sf.net, C...

In [10]:
wwtp_isource_df = isource_to_df(wwtp_fpath)
isource_av_yr_df = calculate_average_annual_sum(wwtp_isource_df, start_year, end_year)
isource_vassom_df = sum_over_vassom(isource_av_yr_df)
wwtp_riv_df = read_wwtp_river_inputs(wwtp_isource_map_fpath, riv_additions_sheet)
tot_wwtw_df = total_wwtw_inputs(isource_vassom_df, wwtp_riv_df)

tot_wwtw_df

Number of WWTPs: 23 


,TOTN,DIN,real_river
Vassom,,,
001.,116.119700,104.344566,6.0
002.,727.099000,653.525220,7.0
003.,290.075179,260.723017,8.0
004.,56.661621,50.928138,9.0
005.,56.560620,49.157339,10.0
006.,394.914328,257.202619,11.0
007.,7.495190,5.494430,12.0
009.,926.376888,744.229333,14.0
011.,94.133032,84.607887,17.0
